# LOCATION ANALYSIS

## Analyzing Neighborhoods in Mumbai, India for Starting a Restaurant

##### PURPOSE OF PROJECT:-   
###### The aim of this project is to study the neighborhoods in Mumbai to determine possible locations for opening a restaurant. This project can be useful for business owners and entrepreneurs who are looking to invest in a restaurant in Mumbai. The main objective of this project is to carefully analyze appropriate data and find recommendations for the stakeholders.

##### DATA COLLECTION:- 
###### Neighborhood Data: The data of the neighborhoods in Mumbai was scraped from wikipedia.The data is read into a pandas data frame using the read_html() method.
###### Geographical Coordinates: The geographical coordinates for Mumbai data has been obtained from the GeoPy library in python. This data is relevant for plotting the map of Mumbai using the Folium library in python. The geocoder library in python has been used to obtain latitude and longitude data for various neighborhoods in Mumbai.
###### Venue Data: The venue data has been extracted using the Foursquare API. This data contains venue recommendations for all neighborhoods in Mumbai and is used to study the popular venues of different neighborhoods.

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.7MB/s 


In [3]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.metrics import silhouette_score

%matplotlib notebook

print('All libraries imported.')

All libraries imported.


Scrapping Neighborhood data from wikipedia

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')[-1]
df.rename(columns={'Area': 'Neighborhood'}, inplace=True)
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270
5,Marol,"Andheri,Western Suburbs",19.119219,72.882743
6,Sahar,"Andheri,Western Suburbs",19.098889,72.867222
7,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018
8,Versova,"Andheri,Western Suburbs",19.120000,72.820000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111


Lets look at the different values for Location present in the Location column.

In [5]:
df['Location'].value_counts()

South Mumbai                       30
Andheri,Western Suburbs             8
Western Suburbs                     6
Eastern Suburbs                     4
Powai,Eastern Suburbs               3
Kandivali West,Western Suburbs      3
Bandra,Western Suburbs              3
Ghatkopar,Eastern Suburbs           3
Mira-Bhayandar,Western Suburbs      3
Malad,Western Suburbs               2
Borivali (West),Western Suburbs     2
Kalbadevi,South Mumbai              2
Vasai,Western Suburbs               2
Mumbai                              2
Goregaon,Western Suburbs            2
Harbour Suburbs                     2
Khar,Western Suburbs                2
Fort,South Mumbai                   1
Antop Hill,South Mumbai             1
Byculla,South Mumbai                1
Kurla,Eastern Suburbs               1
Dadar,South Mumbai                  1
Trombay,Harbour Suburbs             1
Tardeo,South Mumbai                 1
Colaba,South Mumbai                 1
Kandivali East,Western Suburbs      1
Govandi,Harb

We can see that there are many locations that appear only once or twice. This is because the main locations like "Western Suburbs" or "South Mumbai" are being further divided by the area within these locations.

In [6]:
df['Location'] = df['Location'].apply(lambda x: x.split(',')[-1])
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,Western Suburbs,19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,Western Suburbs,19.124085,72.831373
3,Four Bungalows,Western Suburbs,19.124714,72.827210
4,Lokhandwala,Western Suburbs,19.130815,72.829270
5,Marol,Western Suburbs,19.119219,72.882743
6,Sahar,Western Suburbs,19.098889,72.867222
7,Seven Bungalows,Western Suburbs,19.129052,72.817018
8,Versova,Western Suburbs,19.120000,72.820000
9,Mira Road,Western Suburbs,19.284167,72.871111


Now lets again look at the values in Location column.

In [7]:
df['Location'].value_counts()

South Mumbai       39
Western Suburbs    36
Eastern Suburbs    12
Harbour Suburbs     4
Mumbai              2
Name: Location, dtype: int64

In [8]:
df

,Neighborhood,Location,Latitude,Longitude
0,Amboli,Western Suburbs,19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,Western Suburbs,19.124085,72.831373
3,Four Bungalows,Western Suburbs,19.124714,72.827210
4,Lokhandwala,Western Suburbs,19.130815,72.829270
...,...,...,...,...
88,Parel,South Mumbai,18.990000,72.840000
89,Gowalia Tank,South Mumbai,18.962450,72.809703
90,Dava Bazaar,South Mumbai,18.946882,72.831362
91,Dharavi,Mumbai,19.040208,72.850850


Although the data we gathered contained latitude and longitude information, we can reconfirm these coordinates using Geocoder.

In [9]:
df['Latitude1'] = None
df['Longitude1'] = None

for i, neigh in enumerate(df['Neighborhood']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neigh))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    df.loc[i, 'Latitude1'] = latitude
    df.loc[i, 'Longitude1'] = longitude

df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1
0,Amboli,Western Suburbs,19.129300,72.843400,19.1291,72.8464
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,19.1084,72.8623
2,D.N. Nagar,Western Suburbs,19.124085,72.831373,19.1251,72.8325
3,Four Bungalows,Western Suburbs,19.124714,72.827210,19.1264,72.8242
4,Lokhandwala,Western Suburbs,19.130815,72.829270,19.1432,72.8249
5,Marol,Western Suburbs,19.119219,72.882743,19.1191,72.8828
6,Sahar,Western Suburbs,19.098889,72.867222,19.1027,72.8626
7,Seven Bungalows,Western Suburbs,19.129052,72.817018,19.1286,72.8212
8,Versova,Western Suburbs,19.120000,72.820000,19.1377,72.8135
9,Mira Road,Western Suburbs,19.284167,72.871111,19.2656,72.8706


In [10]:
df['Latdiff'] = abs(df['Latitude'] - df['Latitude1'])
df['Longdiff'] = abs(df['Longitude'] - df['Longitude1'])
df.head(10)
df.loc[df.Latdiff>0.001, 'Latitude'] = df.loc[df.Latdiff>0.001, 'Latitude1']
df.loc[df.Longdiff>0.001, 'Longitude'] = df.loc[df.Longdiff>0.001, 'Longitude1']
df.head(10)
df.drop(['Latitude1', 'Longitude1', 'Latdiff', 'Longdiff'], axis=1, inplace=True)
df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.129300,72.843400,19.1291,72.8464,0.00024,0.00304
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833,19.1084,72.8623,0.003028,0.001497
2,D.N. Nagar,Western Suburbs,19.124085,72.831373,19.1251,72.8325,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.124714,72.827210,19.1264,72.8242,0.001666,0.00301
4,Lokhandwala,Western Suburbs,19.130815,72.829270,19.1432,72.8249,0.012345,0.0044
5,Marol,Western Suburbs,19.119219,72.882743,19.1191,72.8828,0.000169,6.7e-05
6,Sahar,Western Suburbs,19.098889,72.867222,19.1027,72.8626,0.00377822,0.00462255
7,Seven Bungalows,Western Suburbs,19.129052,72.817018,19.1286,72.8212,0.000492,0.004162
8,Versova,Western Suburbs,19.120000,72.820000,19.1377,72.8135,0.01769,0.00652
9,Mira Road,Western Suburbs,19.284167,72.871111,19.2656,72.8706,0.0185438,0.000467611


In [11]:
df.loc[df.Latdiff>0.001, 'Latitude'] = df.loc[df.Latdiff>0.001, 'Latitude1']
df.loc[df.Longdiff>0.001, 'Longitude'] = df.loc[df.Longdiff>0.001, 'Longitude1']
df.head(10)

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.1293,72.8464,19.1291,72.8464,0.00024,0.00304
1,"Chakala, Andheri",Western Suburbs,19.1084,72.8623,19.1084,72.8623,0.003028,0.001497
2,D.N. Nagar,Western Suburbs,19.1241,72.8325,19.1251,72.8325,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001666,0.00301
4,Lokhandwala,Western Suburbs,19.1432,72.8249,19.1432,72.8249,0.012345,0.0044
5,Marol,Western Suburbs,19.1192,72.8827,19.1191,72.8828,0.000169,6.7e-05
6,Sahar,Western Suburbs,19.1027,72.8626,19.1027,72.8626,0.00377822,0.00462255
7,Seven Bungalows,Western Suburbs,19.1291,72.8212,19.1286,72.8212,0.000492,0.004162
8,Versova,Western Suburbs,19.1377,72.8135,19.1377,72.8135,0.01769,0.00652
9,Mira Road,Western Suburbs,19.2656,72.8711,19.2656,72.8706,0.0185438,0.000467611


In [12]:
df.where(df['Latitude']==df['Latitude1'])

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Chakala, Andheri",Western Suburbs,19.1084,72.8623,19.1084,72.8623,0.003028,0.001497
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001666,0.00301
4,Lokhandwala,Western Suburbs,19.1432,72.8249,19.1432,72.8249,0.012345,0.0044
...,...,...,...,...,...,...,...,...
88,Parel,South Mumbai,18.9957,72.84,18.9957,72.8391,0.0057,0.00087
89,Gowalia Tank,South Mumbai,18.9645,72.8112,18.9645,72.8112,0.00201,0.001467
90,Dava Bazaar,South Mumbai,19.1314,72.927,19.1314,72.927,0.184518,0.095598
91,Dharavi,Mumbai,19.0467,72.8546,19.0467,72.8546,0.006532,0.00376


In [13]:
df.where(df['Longitude']==df['Longitude1'])

,Neighborhood,Location,Latitude,Longitude,Latitude1,Longitude1,Latdiff,Longdiff
0,Amboli,Western Suburbs,19.1293,72.8464,19.1291,72.8464,0.00024,0.00304
1,"Chakala, Andheri",Western Suburbs,19.1084,72.8623,19.1084,72.8623,0.003028,0.001497
2,D.N. Nagar,Western Suburbs,19.1241,72.8325,19.1251,72.8325,0.000965,0.001107
3,Four Bungalows,Western Suburbs,19.1264,72.8242,19.1264,72.8242,0.001666,0.00301
4,Lokhandwala,Western Suburbs,19.1432,72.8249,19.1432,72.8249,0.012345,0.0044
...,...,...,...,...,...,...,...,...
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Gowalia Tank,South Mumbai,18.9645,72.8112,18.9645,72.8112,0.00201,0.001467
90,Dava Bazaar,South Mumbai,19.1314,72.927,19.1314,72.927,0.184518,0.095598
91,Dharavi,Mumbai,19.0467,72.8546,19.0467,72.8546,0.006532,0.00376


In [14]:
df.drop(['Latitude1', 'Longitude1', 'Latdiff', 'Longdiff'], axis=1, inplace=True)
df.head(10)

,Neighborhood,Location,Latitude,Longitude
0,Amboli,Western Suburbs,19.1293,72.8464
1,"Chakala, Andheri",Western Suburbs,19.1084,72.8623
2,D.N. Nagar,Western Suburbs,19.1241,72.8325
3,Four Bungalows,Western Suburbs,19.1264,72.8242
4,Lokhandwala,Western Suburbs,19.1432,72.8249
5,Marol,Western Suburbs,19.1192,72.8827
6,Sahar,Western Suburbs,19.1027,72.8626
7,Seven Bungalows,Western Suburbs,19.1291,72.8212
8,Versova,Western Suburbs,19.1377,72.8135
9,Mira Road,Western Suburbs,19.2656,72.8711


In [15]:
neighborhoods_mumbai = df.groupby('Location')['Neighborhood'].nunique()
neighborhoods_mumbai

Location
Eastern Suburbs    12
Harbour Suburbs     4
Mumbai              2
South Mumbai       39
Western Suburbs    36
Name: Neighborhood, dtype: int64

 we see one of the locations as Mumbai itself? This is because the neighborhoods contained in this location are located at the outskirts of Mumbai and thus have been grouped as just Mumbai.
 Now lets visualize the neighborhoods on a map using Folium. First we will obtain the geographical coordinates of Mumbai using GeoPy.

In [16]:
address = 'Mumbai, IN'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Mumbai are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Mumbai are 19.0759899, 72.8773928.


In [17]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, location, neighborhood in zip(df['Latitude'], df['Longitude'], df['Location'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mum)  
    
map_mum

Now we can start working with the Foursquare API to obtain venue recommendations.

In [19]:
neighborhood_name = df.loc[0, 'Neighborhood']
neighborhood_lat = df.loc[0, 'Latitude']
neighborhood_long = df.loc[0, 'Longitude']
print("The neighborhood is {} and it's geographical coordinates are {} latitude and {} longitude".format(neighborhood_name,
                                                                                                        neighborhood_lat, neighborhood_long))

The neighborhood is Amboli and it's geographical coordinates are 19.1293 latitude and 72.84644000000003 longitude


In [20]:
CLIENT_ID = '2GQBW5PR0QFXTOGCHKTRFWJBTGOFOHXW1TRTNRAFURQ5FE1X'
CLIENT_SECRET = '3QH40WMZIIDSQN1RFAVAEQHUIMOQUJPKYPABQVNTSDQJN2YD'
VERSION = 20202808
radius = 1000
LIMIT = 200

In [21]:

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius, 
    LIMIT)

use GET method to get the results and convert into json file.

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6098ef6f90c0de109e121a15'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d10d39b7177b1f7d2c75322-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10f941735',
         'name': 'Indian Restaurant',
         'pluralName': 'Indian Restaurants',
         'primary': True,
         'shortName': 'Indian'}],
       'id': '4d10d39b7177b1f7d2c75322',
       'location': {'address': 'S V Road',
        'cc': 'IN',
        'city': 'Mumbai',
        'country': 'India',
        'crossStreet': 'Andheri West',
        'distance': 84,
        'formattedAddress': ['S V Road (Andheri West)',
         'Mumbai 400058',
         'Mahārāshtra',
         'India'],
        'labeledLat

We will now create a function get_category_type to extract the categories of venues.

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we can clean the JSON obtained using the GET method and store our results in a dataframe.

In [24]:
     venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cafe Arfa,Indian Restaurant,19.128930,72.847140
1,"5 Spice , Bandra",Chinese Restaurant,19.130421,72.847206
2,Jaffer Bhai's Delhi Darbar,Mughlai Restaurant,19.137714,72.845909
3,Narayan Sandwich,Sandwich Place,19.121398,72.850270
4,Shawarma Factory,Falafel Restaurant,19.124591,72.840398


We can check how many venues were returned by Foursquare.

In [25]:
print("{} venues were returned for {} by Foursquare".format(len(nearby_venues), neighborhood_name))

27 venues were returned for Amboli by Foursquare


Now that we have seen how the API call works and how we can clean our data to get relevant information, we can generalize this procedure to get nearby venues for all neighborhoods by creating the function getNearbyVenues.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We can apply the function created to get nearby venues for all neighborhoods in Mumbai. We will get 200 nearby venues within a 1km radius, same as before.

In [27]:
mum_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=radius)

Amboli
Chakala, Andheri
D.N. Nagar
Four Bungalows
Lokhandwala
Marol
Sahar
Seven Bungalows
Versova
Mira Road
Bhayandar
Uttan
Bandstand Promenade
Kherwadi
Pali Hill
I.C. Colony
Gorai
Dahisar
Aarey Milk Colony
Bangur Nagar
Jogeshwari West
Juhu
Charkop
Poisar
Mahavir Nagar
Thakur village
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Kalina
Naigaon
Nalasopara
Virar
Irla
Vile Parle
Bhandup
Amrut Nagar
Asalfa
Pant Nagar
Kanjurmarg
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Indian Institute of Technology Bombay campus
Vidyavihar
Vikhroli
Chembur
Deonar
Mankhurd
Mahul
Agripada
Altamount Road
Bhuleshwar
Breach Candy
Carmichael Road
Cavel
Churchgate
Cotton Green
Cuffe Parade
Cumbala Hill
Currey Road
Dhobitalao
Dongri
Kala Ghoda
Kemps Corner
Lower Parel
Mahalaxmi
Mahim
Malabar Hill
Marine Drive
Marine Lines
Mumbai Central
Nariman Point
Prabhadevi
Sion
Walkeshwar
Worli
C.G.S. colony
Dagdi Chawl
Navy Nagar
Hindu colony
Ballard Estate
Chira Bazaar
Fanas Wadi
Chor Bazaar
Matunga
Parel
Gowalia Tank


Lets see what our dataframe looks like.

In [28]:
print(mum_venues.shape)
mum_venues.head(10)

(3528, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.84644,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,Amboli,19.1293,72.84644,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
2,Amboli,19.1293,72.84644,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant
3,Amboli,19.1293,72.84644,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Amboli,19.1293,72.84644,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
5,Amboli,19.1293,72.84644,Persia Darbar,19.136952,72.846822,Indian Restaurant
6,Amboli,19.1293,72.84644,Sarvodaya Veg. Restaurant,19.123760,72.850893,Indian Restaurant
7,Amboli,19.1293,72.84644,Domino's Pizza,19.131000,72.848000,Pizza Place
8,Amboli,19.1293,72.84644,Garden Court,19.127188,72.837478,Indian Restaurant
9,Amboli,19.1293,72.84644,Subway,19.127860,72.844461,Sandwich Place


Lets see how many venues were returned for each neighborhood.

In [29]:
mum_venues.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aarey Milk Colony,12,12,12,12,12,12
1,Agripada,29,29,29,29,29,29
2,Altamount Road,68,68,68,68,68,68
3,Amboli,27,27,27,27,27,27
4,Amrut Nagar,12,12,12,12,12,12
...,...,...,...,...,...,...,...
88,Vikhroli,10,10,10,10,10,10
89,Vile Parle,78,78,78,78,78,78
90,Virar,11,11,11,11,11,11
91,Walkeshwar,13,13,13,13,13,13


Lets see what our dataframe looks like.

In [30]:
print("There are {} unique categories".format(mum_venues['Venue Category'].nunique()))

There are 219 unique categories


We can start analyzing each neighborhood by One-hot Encoding to see which categories belong in which neighborhoods.

In [31]:
# one-hot encoding
mum_onehot = pd.get_dummies(mum_venues[['Venue Category']], prefix="", prefix_sep="")
mum_onehot.head()

,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,...,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Adding Neighborhood column to the one-hot encoded dataframe.

In [32]:
mum_onehot['Neighborhood'] = mum_venues['Neighborhood']
mum_onehot.head()

,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,...,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Moving the Neighborhood column to the first column.

In [33]:
temp = list(mum_onehot.columns)

if 'Neighborhood' in temp:
    temp.remove('Neighborhood')
    
fixed_columns = ['Neighborhood'] + temp
mum_onehot = mum_onehot[fixed_columns]

mum_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,...,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now we can groupby neighborhood and take the mean for all categories.

In [34]:
mum_grouped = mum_onehot.groupby('Neighborhood', sort=False).mean().reset_index()
print(mum_grouped.shape)
mum_grouped.head(10)

(93, 219)


,Neighborhood,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,...,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Amboli,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.074074,0.037037,0.0,0.0,0.0,0.037037,0.0,0.074074,0.0000,0.000000,0.0,0.000000,0.0,0.037037,0.000000,0.000000,0.0,0.0,0.037037,0.0,0.0,0.000000,0.0,0.0,0.000000,0.037037,0.0,0.037037,0.0,...,0.074074,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.037037,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.037037,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Chakala, Andheri",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.046512,0.000000,0.0,0.0,0.0,0.000000,0.0,0.046512,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.023256,0.0,0.000000,0.0,...,0.000000,0.0,0.023256,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.023256,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.046512,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,D.N. Nagar,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.021739,0.000000,0.0,0.0,0.0,0.000000,0.0,0.130435,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.021739,0.0,0.0,0.000000,0.0,0.0,0.000000,0.021739,0.0,0.000000,0.0,...,0.021739,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.043478,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.021739,0.0,0.0,0.000000,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.043478,0.0,0.0
3,Four Bungalows,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.050000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.016667,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.050000,0.016667,0.0,0.0000,0.0,0.016667,0.0,0.0,0.0,0.016667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.016667,0.0,0.0
4,Lokhandwala,0.0,0.0,0.0,0.0,0.011236,0.0,0.0,0.0,0.0,0.000000,0.022472,0.000000,0.0,0.0,0.0,0.022472,0.0,0.033708,0.0000,0.000000,0.0,0.022472,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.022472,0.0,0.0,0.011236,0.000000,0.0,0.000000,0.0,...,0.011236,0.0,0.022472,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.011236,0.011236,0.0,0.011236,0.0,0.0,0.011236,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.011236,0.0,0.0,0.0,0.011236,0.0,0.0,0.0,0.0,0.011236,0.0,0.0
5,Marol,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,Sa

In order to further understand the data, we can display the top 5 venues of all neighborhoods.

In [35]:
num_top_venues = 5

for hood in mum_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mum_grouped[mum_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amboli----
               venue  freq
0  Indian Restaurant  0.15
1     Sandwich Place  0.07
2        Pizza Place  0.07
3   Asian Restaurant  0.07
4                Bar  0.07


----Chakala, Andheri----
                           venue  freq
0                          Hotel  0.16
1              Indian Restaurant  0.12
2                           Café  0.12
3                      Hotel Bar  0.05
4  Vegetarian / Vegan Restaurant  0.05


----D.N. Nagar----
                  venue  freq
0                   Bar  0.13
1                   Pub  0.09
2  Gym / Fitness Center  0.07
3     Indian Restaurant  0.07
4           Pizza Place  0.07


----Four Bungalows----
                venue  freq
0                 Pub  0.07
1   Indian Restaurant  0.07
2                Café  0.07
3  Seafood Restaurant  0.05
4  Chinese Restaurant  0.05


----Lokhandwala----
                venue  freq
0   Indian Restaurant  0.11
1  Chinese Restaurant  0.07
2                Café  0.06
3                 Pub  0.04
4     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


            venue  freq
0            Café  0.08
1          Bakery  0.08
2  Sandwich Place  0.06
3             Bar  0.06
4            Park  0.04


----Carmichael Road----
                           venue  freq
0                            Bar  0.07
1             Chinese Restaurant  0.07
2             Italian Restaurant  0.04
3  Vegetarian / Vegan Restaurant  0.04
4                         Bakery  0.04


----Cavel----
                  venue  freq
0     Indian Restaurant  0.25
1                  Café  0.06
2  Fast Food Restaurant  0.04
3            Restaurant  0.04
4                Bakery  0.04


----Churchgate----
                  venue  freq
0     Indian Restaurant  0.14
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.06
3                  Café  0.06
4                   Bar  0.05


----Cotton Green----
                  venue  freq
0                 Plaza  0.29
1           Pizza Place  0.14
2         Train Station  0.14
3  Fast Food Restaurant  0.14
4             Multiplex  0

Lets now create a dataframe with the top 10 common venues for each neighborhood.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mum_grouped['Neighborhood']

for ind in np.arange(mum_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mum_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Indian Restaurant,Pizza Place,Sandwich Place,Asian Restaurant,Bar,Bakery,Burger Joint,Snack Place,Bowling Alley,Bus Station
1,"Chakala, Andheri",Hotel,Indian Restaurant,Café,Asian Restaurant,Hotel Bar,Fast Food Restaurant,Multiplex,Restaurant,Bar,Pizza Place
2,D.N. Nagar,Bar,Pub,Pizza Place,Gym / Fitness Center,Indian Restaurant,Women's Store,Juice Bar,Vegetarian / Vegan Restaurant,Lounge,Snack Place
3,Four Bungalows,Pub,Café,Indian Restaurant,Lounge,Seafood Restaurant,Ice Cream Shop,Bar,Chinese Restaurant,Coffee Shop,Department Store
4,Lokhandwala,Indian Restaurant,Chinese Restaurant,Café,Pub,Italian Restaurant,Bar,Coffee Shop,Gym / Fitness Center,Seafood Restaurant,Bakery
...,...,...,...,...,...,...,...,...,...,...,...
88,Parel,Coffee Shop,Indian Restaurant,Roof Deck,Restaurant,Lounge,Cafeteria,Chinese Restaurant,Maharashtrian Restaurant,Bus Station,Hotel
89,Gowalia Tank,Indian Restaurant,Café,Fast Food Restaurant,Bakery,Coffee Shop,Snack Place,Chinese Restaurant,Sandwich Place,Pizza Place,Electronics Store
90,Dava Bazaar,Train Station,Hotel,Smoke Shop,Fast Food Restaurant,Café,Restaurant,Asian Restaurant,Fish Market,Cupcake Shop,Coffee Shop
91,Dharavi,Indian Restaurant,Gym / Fitness Center,Lake,Shoe Store,Fast Food Restaurant,Seafood Restaurant,Food & Drink Shop,Market,Sandwich Place,Luggage Store


# Clustering neighborhoods

Now we can use KMeans clustering method to cluster the neighborhoods.

First we need to determine how many clusters to use. This will be done using the Silhouette Score.

We will define a function to plot the Silhouette Score that will be calculated using different number of clusters.

In [39]:
def plot(x, y):
    fig = plt.figure(figsize=(12,6))
    plt.plot(x, y, 'o-')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Scores')
    plt.title('Checking Optimum Number of Clusters')
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [40]:
maxk = 15
scores = []
kval = []

for k in range(2, maxk+1):
    cl_df = mum_grouped.drop('Neighborhood', axis=1)
    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=40).fit_predict(cl_df) #Choose any random_state
    
    score = silhouette_score(cl_df, kmeans, metric='euclidean', random_state=0)
    kval.append(k)
    scores.append(score)
    
print(scores)
print(kval)
plt.plot(kval, scores)
plt.show()
k = 5

mum_clustering = mum_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=k, init="k-means++", random_state=40).fit(mum_clustering) #Can choose any random_state

kmeans.labels_

We can now display the scores for different number of clusters and plot the data as well.

In [41]:
print(scores)
print(kval)
plt.plot(kval, scores)
plt.show()

[0.10369510534392831, 0.09192987577576635, 0.11468626159016908, 0.10674722777057133, 0.10727193946633932, 0.06316035617995194, 0.08749221368088794, 0.08932176586397123, 0.08437597805371402, 0.08464495757957764, 0.07542051867441553, 0.043440909179432975, 0.06606186133603253, 0.045152237071555]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


<IPython.core.display.Javascript object>

We can see that the silhouette scores are not very high even as we increase the number of clusters. This means that the inter-cluster distance between different clusters is not very high over the range of k-values. However, we will try to cluster our data as best as we can. For this, we will use 5 clusters for our clustering model since it provides the highest silhouette score as seen above.

In [42]:
k = 5

mum_clustering = mum_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=k, init="k-means++", random_state=40).fit(mum_clustering) #Can choose any random_state

kmeans.labels_

array([3, 2, 2, 2, 2, 3, 2, 2, 2, 3, 1, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3,
       2, 1, 2, 2, 2, 2, 2, 3, 2, 3, 0, 2, 3, 2, 1, 3, 2, 3, 1, 2, 1, 3,
       2, 3, 3, 2, 3, 2, 1, 3, 3, 2, 3, 2, 2, 3, 2, 0, 3, 2, 2, 3, 3, 2,
       2, 2, 3, 3, 4, 3, 3, 2, 2, 2, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3,
       3, 2, 1, 3, 3], dtype=int32)

Now we can create a new dataframe that includes cluster labels and the top 10 venues.

In [43]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
mum_merged = df
mum_merged = mum_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
print(mum_merged.shape)
mum_merged
mum_merged.loc[mum_merged['Cluster Labels'] == 0, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]
mum_merged.loc[mum_merged['Cluster Labels'] == 1, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]
mum_merged.loc[mum_merged['Cluster Labels'] == 2, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]
mum_merged.loc[mum_merged['Cluster Labels'] == 3, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]
mum_merged.loc[mum_merged['Cluster Labels'] == 4, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

Lets view the newly created dataframe.

In [44]:
print(mum_merged.shape)
mum_merged

(93, 15)


,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Western Suburbs,19.1293,72.8464,3,Indian Restaurant,Pizza Place,Sandwich Place,Asian Restaurant,Bar,Bakery,Burger Joint,Snack Place,Bowling Alley,Bus Station
1,"Chakala, Andheri",Western Suburbs,19.1084,72.8623,2,Hotel,Indian Restaurant,Café,Asian Restaurant,Hotel Bar,Fast Food Restaurant,Multiplex,Restaurant,Bar,Pizza Place
2,D.N. Nagar,Western Suburbs,19.1241,72.8325,2,Bar,Pub,Pizza Place,Gym / Fitness Center,Indian Restaurant,Women's Store,Juice Bar,Vegetarian / Vegan Restaurant,Lounge,Snack Place
3,Four Bungalows,Western Suburbs,19.1264,72.8242,2,Pub,Café,Indian Restaurant,Lounge,Seafood Restaurant,Ice Cream Shop,Bar,Chinese Restaurant,Coffee Shop,Department Store
4,Lokhandwala,Western Suburbs,19.1432,72.8249,2,Indian Restaurant,Chinese Restaurant,Café,Pub,Italian Restaurant,Bar,Coffee Shop,Gym / Fitness Center,Seafood Restaurant,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Parel,South Mumbai,18.9957,72.84,3,Coffee Shop,Indian Restaurant,Roof Deck,Restaurant,Lounge,Cafeteria,Chinese Restaurant,Maharashtrian Restaurant,Bus Station,Hotel
89,Gowalia Tank,South Mumbai,18.9645,72.8112,2,Indian Restaurant,Café,Fast Food Restaurant,Bakery,Coffee Shop,Snack Place,Chinese Restaurant,Sandwich Place,Pizza Place,Electronics Store
90,Dava Bazaar,South Mumbai,19.1314,72.927,1,Train Station,Hotel,Smoke Shop,Fast Food Restaurant,Café,Restaurant,Asian Restaurant,Fish Market,Cupcake Shop,Coffee Shop
91,Dharavi,Mumbai,19.0467,72.8546,3,Indian Restaurant,Gym / Fitness Center,Lake,Shoe Store,Fast Food Restaurant,Seafood Restaurant,Food & Drink Shop,Market,Sandwich Place,Luggage Store


We can visualize the clustering by creating a map.

In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can now view the neighborhoods in each cluster and their top 10 most common venues.

# cluster-1


In [46]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Nalasopara,Western Suburbs,Multiplex,Ice Cream Shop,Diner,Pizza Place,Fast Food Restaurant,Electronics Store,Bakery,Donut Shop,Fish Market,Fish & Chips Shop
59,Cotton Green,South Mumbai,Plaza,Pizza Place,Multiplex,Fast Food Restaurant,Train Station,Bakery,Fish & Chips Shop,Field,Farmers Market,Falafel Restaurant


# cluster-2

In [47]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Bhayandar,Western Suburbs,Ice Cream Shop,Pizza Place,Indian Restaurant,Food Truck,Diner,Bakery,Train Station,Soccer Field,Platform,Cosmetics Shop
23,Poisar,Western Suburbs,Mexican Restaurant,Indian Restaurant,Bus Line,Snack Place,Recreation Center,Sporting Goods Shop,Train Station,Electronics Store,Park,Gym / Fitness Center
36,Bhandup,Eastern Suburbs,Train Station,Ice Cream Shop,Indian Restaurant,Asian Restaurant,Bakery,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Event Space,Factory
40,Kanjurmarg,Eastern Suburbs,Train Station,Hotel,Gym,Gift Shop,Asian Restaurant,Chinese Restaurant,Multiplex,Cupcake Shop,Fast Food Restaurant,Donut Shop
42,Nahur,Eastern Suburbs,Restaurant,Train Station,Pizza Place,Fast Food Restaurant,Coffee Shop,Bus Station,Pub,Event Space,Donut Shop,Electronics Store
50,Mankhurd,Harbour Suburbs,Coffee Shop,Sports Bar,Train Station,Bus Station,Zoo,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
90,Dava Bazaar,South Mumbai,Train Station,Hotel,Smoke Shop,Fast Food Restaurant,Café,Restaurant,Asian Restaurant,Fish Market,Cupcake Shop,Coffee Shop


# cluster-3

In [48]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Chakala, Andheri",Western Suburbs,Hotel,Indian Restaurant,Café,Asian Restaurant,Hotel Bar,Fast Food Restaurant,Multiplex,Restaurant,Bar,Pizza Place
2,D.N. Nagar,Western Suburbs,Bar,Pub,Pizza Place,Gym / Fitness Center,Indian Restaurant,Women's Store,Juice Bar,Vegetarian / Vegan Restaurant,Lounge,Snack Place
3,Four Bungalows,Western Suburbs,Pub,Café,Indian Restaurant,Lounge,Seafood Restaurant,Ice Cream Shop,Bar,Chinese Restaurant,Coffee Shop,Department Store
4,Lokhandwala,Western Suburbs,Indian Restaurant,Chinese Restaurant,Café,Pub,Italian Restaurant,Bar,Coffee Shop,Gym / Fitness Center,Seafood Restaurant,Bakery
6,Sahar,Western Suburbs,Hotel,Indian Restaurant,Café,Restaurant,Lounge,Bar,Asian Restaurant,Convenience Store,Seafood Restaurant,Coffee Shop
7,Seven Bungalows,Western Suburbs,Café,Bar,Pub,Ice Cream Shop,Pizza Place,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop
8,Versova,Western Suburbs,Café,Beach,Ice Cream Shop,Coffee Shop,Chinese Restaurant,Pizza Place,Sandwich Place,Restaurant,Recreation Center,Pub
12,Bandstand Promenade,Western Suburbs,Coffee Shop,Café,Indian Restaurant,Scenic Lookout,Tea Room,Deli / Bodega,Performing Arts Venue,Cocktail Bar,Department Store,Dessert Shop
13,Kherwadi,Western Suburbs,Indian Restaurant,Café,Italian Restaurant,Restaurant,Seafood Restaurant,Bar,Hookah Bar,Food Court,Lake,Spa
14,Pali Hill,Western Suburbs,Indian Restaurant,Bakery,Dessert Shop,Café,Fast Food Restaurant,Italian Restaurant,Seafood Restaurant,Asian Restaurant,Chinese Restaurant,Ice Cream Shop


# cluster -4

In [49]:
mum_merged.loc[mum_merged['Cluster Labels'] == 3, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amboli,Western Suburbs,Indian Restaurant,Pizza Place,Sandwich Place,Asian Restaurant,Bar,Bakery,Burger Joint,Snack Place,Bowling Alley,Bus Station
5,Marol,Western Suburbs,Indian Restaurant,Hotel,Coffee Shop,Boat or Ferry,Fast Food Restaurant,Lounge,Diner,Chinese Restaurant,Restaurant,Ice Cream Shop
9,Mira Road,Western Suburbs,Indian Restaurant,Shopping Mall,Coffee Shop,Movie Theater,General College & University,Gift Shop,Food Truck,Basketball Court,Convenience Store,Vegetarian / Vegan Restaurant
11,Uttan,Western Suburbs,Convenience Store,Whisky Bar,Beach,Indian Restaurant,Restaurant,Creperie,Cosmetics Shop,Fish Market,Fish & Chips Shop,Field
15,I.C. Colony,Western Suburbs,Indian Restaurant,Chinese Restaurant,Convenience Store,Coffee Shop,Fast Food Restaurant,Bar,Bakery,Pizza Place,BBQ Joint,Department Store
20,Jogeshwari West,Western Suburbs,Indian Restaurant,Snack Place,Mughlai Restaurant,Chinese Restaurant,Asian Restaurant,Men's Store,Café,Gym,Ice Cream Shop,Field
21,Juhu,Western Suburbs,Indian Restaurant,Coffee Shop,Restaurant,Café,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Movie Theater,Hotel,Pizza Place,Buffet
29,Sunder Nagar,Western Suburbs,Indian Restaurant,Coffee Shop,Bakery,Fast Food Restaurant,Café,Shopping Mall,Breakfast Spot,Restaurant,Movie Theater,Pizza Place
31,Naigaon,Western Suburbs,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Café,Juice Bar,Farmers Market,Breakfast Spot,Snack Place,Convention Center
34,Irla,Western Suburbs,Indian Restaurant,Coffee Shop,Café,Bakery,Chinese Restaurant,Electronics Store,Pizza Place,Clothing Store,Dessert Shop,Sandwich Place


# cluster-5

In [50]:
mum_merged.loc[mum_merged['Cluster Labels'] == 4, mum_merged.columns[[0] + [1] + list(range(5, mum_merged.shape[1]))]]

,Neighborhood,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Malabar Hill,South Mumbai,Ice Cream Shop,Park,Coffee Shop,Food Truck,Indian Restaurant,Lighthouse,Dessert Shop,Farmers Market,Event Space,Factory
77,Walkeshwar,South Mumbai,Park,Food Truck,Ice Cream Shop,Food & Drink Shop,Indian Restaurant,Coffee Shop,Lighthouse,Restaurant,Dessert Shop,Factory
81,Navy Nagar,South Mumbai,Ice Cream Shop,Park,Italian Restaurant,Asian Restaurant,Restaurant,Chinese Restaurant,Department Store,Basketball Court,Pizza Place,Farmers Market


# Result

By analyzing the five clusters obtained we can see that some of the clusters are more suited for restaurants and hotels, whereas, other clusters are less suited. Neighborhoods in clusters 3, 4, and 5 contain a small percentage of restaurants, hotels, cafe and pubs in their top 10 common venues. These clusters contain a higher degree of other venues like train station, bus station, fish market, gym, performing arts venue and smoke shop, to name a few. Thus, they are not well suited for opening a new restaurant. On the other hand, neighborhoods in clusters 1 and 2 contain a much higher degree of restaurants, hotels, multiplex, cafes, bars and other food joints. Thus, the neighborhoods in these clusters would be well suited for opening a new restaurant.

Comparing clusters 1 and 2, neighborhoods in cluster 1 seem to be more suited for starting a restaurant since they contains a larger percentage of food joints in the top 10 most common venues than cluster 2. The neighborhoods in cluster 1 contain a variety of food joints like restaurants, tea rooms, bakery, cafe, steakhouse and pubs and also contain very diverse cuisines like Japanese, Indian, Chinese, Italian and seafood restaurants. Most neighborhoods in cluster 2 seem to have Indian Restaurant as their top most common venue; however, on careful analysis we can see that neighborhoods in cluster 2 also contain other venues like soccer field, flea market, smoke shop, gym, train station, dance studio, music store, cosmetics shop and so on. Thus, it is recommended that the new restaurant can be opened in the neighborhoods belonging to cluster 1. This neighborhood can be further plotted on a map as shown below.

In [51]:
new_restaurant_neighborhoods = mum_merged.loc[mum_merged['Cluster Labels'] == 0, mum_merged.columns[[0, 1, 2, 3] + list(range(5, mum_merged.shape[1]))]]
new_restaurant_neighborhoods.head()

,Neighborhood,Location,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Nalasopara,Western Suburbs,19.42,72.8141,Multiplex,Ice Cream Shop,Diner,Pizza Place,Fast Food Restaurant,Electronics Store,Bakery,Donut Shop,Fish Market,Fish & Chips Shop
59,Cotton Green,South Mumbai,18.9862,72.8412,Plaza,Pizza Place,Multiplex,Fast Food Restaurant,Train Station,Bakery,Fish & Chips Shop,Field,Farmers Market,Falafel Restaurant


In [52]:
map_res_locations = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, location, neighborhood in zip(new_restaurant_neighborhoods['Latitude'], new_restaurant_neighborhoods['Longitude'],
                                            new_restaurant_neighborhoods['Location'], new_restaurant_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, location)
    folium.Marker([lat, lng], popup='{} has geographical coordinates ({:.4f}, {:.4f})'.format(label, lat, lng),
                  icon=folium.Icon(color='lightred'), tooltip=label).add_to(map_res_locations)
    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_res_locations) 

map_res_locations

# Conclusion

We have successfully analyzed the neighborhoods in Mumbai, India for determining which would be the best neighborhoods for opening a new restaurant. Based on our analysis, neighborhoods in cluster 1 are recommended as locations for the new restaurant. This has also been plotted in the map above.